In [ ]:
╔═══════════════════════════════════════════════════════════════════════════════╗
║                                                                               ║
║   ██████╗ ███████╗███████╗████████╗███████╗██████╗     ██╗   ██╗██╗  ██╗      ║
║   ██╔══██╗██╔════╝██╔════╝╚══██╔══╝██╔════╝██╔══██╗    ██║   ██║██║  ██║      ║
║   ██████╔╝█████╗  ███████╗   ██║   █████╗  ██████╔╝    ██║   ██║███████║      ║
║   ██╔═══╝ ██╔══╝  ╚════██║   ██║   ██╔══╝  ██╔══██╗    ╚██╗ ██╔╝╚════██║      ║
║   ██║     ███████╗███████║   ██║   ███████╗██║  ██║     ╚████╔╝      ██║      ║
║   ╚═╝     ╚══════╝╚══════╝   ╚═╝   ╚══════╝╚═╝  ╚═╝      ╚═══╝       ╚═╝      ║
║                                                                               ║
║                                    ↓↓↓                                        ║
║                                                                               ║
║   ██████╗ ███████╗███████╗████████╗███████╗██████╗     ██╗   ██╗███████╗      ║
║   ██╔══██╗██╔════╝██╔════╝╚══██╔══╝██╔════╝██╔══██╗    ██║   ██║██╔════╝      ║
║   ██████╔╝█████╗  ███████╗   ██║   █████╗  ██████╔╝    ██║   ██║███████╗      ║
║   ██╔═══╝ ██╔══╝  ╚════██║   ██║   ██╔══╝  ██╔══██╗    ╚██╗ ██╔╝╚════██║      ║
║   ██║     ███████╗███████║   ██║   ███████╗██║  ██║     ╚████╔╝ ███████║      ║
║   ╚═╝     ╚══════╝╚══════╝   ╚═╝   ╚══════╝╚═╝  ╚═╝      ╚═══╝  ╚══════╝      ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝

    PSConf.EU 2025 - Chrissy LeMaire (@funbucket.dev)
    Converting 3500+ Pester Tests from v4 to v5 Using AI

In [ ]:
# Since it's coming from my repo and not the Gallery
Import-Module ./aitools.psd1 -Force

# Set default params and create reusable function
$PSDefaultParameterValues["Update-PesterTest:Raw"] = $true
$PSDefaultParameterValues["Start-Process:NoNewWindow"] = $true

# Create function to clear changes
function Clear-Modified {
    Start-Process git -ArgumentLIst @("-C", "../dbatools", "restore", ".")
}

### Check out the 3 integral prompt files


<dl>
  <dt>📄 prompt.md</dt>
  <dt>📄 migration.md</dt>
  <dt>📄 style.md</dt>
</dl>

Now let's see it in action..

#### 🧪 The actual test file

In [ ]:
# Pick a medium-complexity test file
$testFile = "C:\github\dbatools\tests\Invoke-DbaDbShrink.Tests.ps1"

# Show the BEFORE state - classic Pester v4
Start-Process code -ArgumentList $testFile

### 🚀 Migrate with Claude Code

First, check out the command and the tests that couldn't make the cut

In [ ]:
# 📄 Load up public/Update-PesterTest.
Start-Process explorer -ArgumentList C:\github\aitools\public

# 📁 Load up large files
Start-Process explorer -ArgumentList C:\github\dbatools\tests

#### 🐎 Now execute

In [ ]:
# Set Claude Code just in case
Set-AIToolDefault -Tool Claude

# Clear all changes in the dbatools repo
Clear-Modified

# Run Claude
Get-ChildItem C:\github\dbatools\tests\*.Tests.ps1 |
Update-PesterTest -First 1 -Skip 1 -Verbose


### Claude excelled at

<dl>
  <dt>✅ Structural refactoring (BeforeAll/AfterAll)</dt>
  <dt>✅ Parameter test refactoring</dt>
  <dt>✅ Where-Object conversions</dt>
  <dt>✅ Hashtable alignment</dt>
  <dt>✅ Comment preservation</dt>
</dl>

### Claude wasn't as good at

<dl>
  <dt>❌ Large files</dt>
  <dt>❌ While rare, instructions were sometimes missed</dt>
  <dt>❌ Variable scoping fixes</dt>
</dl>

> I would say [Claude migrated] about 80 percent automatically, 10 percent classic manual search and replace and 10 percent hard manual work. Most manual cases were because code was not good in the first place. ~ Andreas Jordan, co-migrator


### Upgrading to v6

1. Update migration.md with v5 → v6 changes
1. Update style.md if needed
1. `Get-ChildItem *.Tests.ps1 | Update-PesterTest`

### ✨ Gemini

In [ ]:
# Clear all changes in the dbatools repo
Clear-Modified

# Set Gemini for that free 1000
Set-AIToolDefault -Tool Gemini

# Run Gemini CLI
Get-ChildItem C:\github\dbatools\tests\*.Tests.ps1 |
Update-PesterTest -First 1 -Skip 1 -Verbose

### 🧭 GitHub Copilot

In [ ]:
# Requires subscription
Set-AIToolDefault -Tool Copilot

#### 🧠 First with GPT-5

In [ ]:
# Clear all changes in the dbatools repo
Clear-Modified

# Update reasoning effort for ...different.. results
Set-AIToolConfig -Tool Codex -ReasoningEffort high

# Run conversion
Get-ChildItem C:\github\dbatools\tests\*DbaDbShrink*.Tests.ps1 |
Update-PesterTest -First 1 -Model gpt-5

##### 🧠 Now with Sonnet

In [ ]:
# Clear all changes in the dbatools repo
Clear-Modified

Get-ChildItem C:\github\dbatools\tests\*DbaDbShrink*.Tests.ps1 |
Update-PesterTest -First 1 -Model claude-sonnet-4.5

### This is repeatable for other types of refactoring 🎉

<dl>
  <dt>🧪 Test frameworks (Jest → Vitest)</dt>
  <dt>🔄 API upgrades (any v1 → v2)</dt>
  <dt>🎨 Linting/style fixes</dt>
  <dt>📝 Documentation updates</dt>
  <dt>🌍 i18n sweeps</dt>
  <dt>🧩 Dependency migrations</dt>
</dl>